In [1]:
import sys
#import click
import numpy as np
import pandas as pd
import rpy2.robjects as ro
#from rpy2.rinterface import RRuntimeError
from rpy2.robjects import pandas2ri, Formula
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
from statsmodels.stats.multitest import multipletests
from rpy2.robjects import pandas2ri

import os
os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Resources/R'

In [2]:
!which R

/usr/local/bin/R


In [2]:
#Import R libraries
base = importr('base')
stats = importr('stats')
limma = importr('limma')
writexl = importr('writexl')
edgeR = importr('edgeR')

In [3]:
#data = data.set_index('ID') #replace 'ID' with your own annotation if necessary
design = pd.read_csv('../data/test_metadata.csv', index_col=0) #replace with your own design file
design = design.loc[:, ["condition"]] #remove first column
design

,condition
sample1,A
sample2,A
sample3,A
sample4,A
sample5,A
...,...
sample96,B
sample97,B
sample98,B
sample99,B


In [4]:
data =  pd.read_csv('../data/test_counts.csv', index_col=0)
#data = data.T #replace your own data file
data

,sample1,sample2,sample3,sample4,sample5,sample6,sample7,sample8,sample9,sample10,...,sample91,sample92,sample93,sample94,sample95,sample96,sample97,sample98,sample99,sample100
gene1,12,1,4,1,1,0,0,2,5,18,...,0,9,5,27,2,7,1,10,18,21
gene2,21,44,4,10,11,8,14,19,19,11,...,31,43,13,12,24,26,14,36,14,9
gene3,4,2,11,2,6,2,17,11,9,8,...,11,4,12,0,1,3,3,2,3,3
gene4,130,63,180,100,135,36,159,60,66,157,...,119,86,112,89,73,67,71,72,66,42
gene5,18,11,21,44,16,26,11,19,7,35,...,50,41,21,86,31,11,33,11,53,13
gene6,0,10,3,9,2,3,19,2,3,0,...,10,0,3,0,12,4,5,2,11,13
gene7,16,70,28,28,32,36,39,41,13,31,...,4,20,31,18,27,41,19,66,32,19
gene8,54,32,34,16,29,42,22,83,25,52,...,28,32,22,39,33,44,42,27,19,78
gene9,49,57,65,33,31,42,38,46,52,37,...,15,45,24,72,67,54,25,16,79,30
gene10,3,9,2,9,5,17,6,12,3,10,...,13,1,31,28,4,1,4,9,11,5


In [38]:
# Convert data and design pandas dataframes to R dataframes
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data = ro.conversion.py2rpy(data)
    r_design = ro.conversion.py2rpy(design)
    # Use the genes index column from data as a R String Vector
    genes = ro.StrVector(
        [
            str(index)
            #added tovalues to convert to numpy array
            for index in data.index.tolist()
            #for index in data.index.tolist()
        ]
    )

In [100]:
print(r_data)

       sample1 sample2 sample3 sample4 sample5 sample6 sample7 sample8 sample9
gene1       12       1       4       1       1       0       0       2       5
gene2       21      44       4      10      11       8      14      19      19
gene3        4       2      11       2       6       2      17      11       9
gene4      130      63     180     100     135      36     159      60      66
gene5       18      11      21      44      16      26      11      19       7
gene6        0      10       3       9       2       3      19       2       3
gene7       16      70      28      28      32      36      39      41      13
gene8       54      32      34      16      29      42      22      83      25
gene9       49      57      65      33      31      42      38      46      52
gene10       3       9       2       9       5      17       6      12       3
       sample10 sample11 sample12 sample13 sample14 sample15 sample16 sample17
gene1        18        3        7        3       23 

In [39]:
# Create a model matrix using design's Target column using the R formula "~0 + f" to get all the unique factors as columns
f = base.factor(r_design.rx2('condition'), levels=base.unique(r_design.rx2('condition')))
form = Formula('~0 + f')
form.environment['f'] = f
r_design = stats.model_matrix(form) #create one-hot matrix
r_design.colnames = base.levels(f) # set column names to unique factors

In [72]:
d0 = edgeR.DGEList(r_data)
d0 = edgeR.calcNormFactors(d0)
y = limma.voom(d0, r_design)


In [73]:
#pandas2ri.activate()
y = y[1]
y

array([[14.91021147, 12.57100385, 13.5372095 , 12.46720801, 12.71370164,
        11.16487439, 10.24889641, 13.44199667, 14.77144307, 15.28411113,
        13.93774719, 14.27957764, 13.3292827 , 16.25287634, 14.78441287,
        16.86777674, 12.44704502, 16.1669913 , 13.05158052, 12.35140264,
        13.29585792, 14.73917623, 13.41823183, 15.92532391, 12.44942293,
        15.65544618, 10.96655316, 14.49047363, 12.17618066, 12.14818673,
        14.61590713, 10.55368513, 14.74008435, 14.21008995, 13.25865272,
        13.04859717, 13.42542844, 10.51489731, 13.03060319, 15.35681546,
        13.02894832, 15.16811392, 13.30931818, 15.71126768, 12.46497481,
        11.5472417 , 15.18891314, 12.84254488, 16.12940973, 13.6184124 ,
        15.15082929, 15.12568065, 15.65654629, 15.01554356, 14.28963739,
        13.88295368, 14.60171204, 16.44066899, 15.62221446, 13.70753691,
        14.45908393, 13.19262747, 14.95889939, 15.73904955, 14.23237343,
        15.5752764 , 15.80043377, 14.39168166, 13.2

In [74]:
y = pd.DataFrame(y,columns=design.index, index=data.index)


In [75]:
y

,sample1,sample2,sample3,sample4,sample5,sample6,sample7,sample8,sample9,sample10,...,sample91,sample92,sample93,sample94,sample95,sample96,sample97,sample98,sample99,sample100
gene1,14.910211,12.571004,13.537209,12.467208,12.713702,11.164874,10.248896,13.441997,14.771443,15.284111,...,11.122322,14.908812,14.148508,15.996024,13.076084,14.913583,12.692114,15.496437,15.889057,16.686187
gene2,15.692620,17.461775,13.537209,15.274563,15.652301,15.252337,15.106877,16.405471,16.597414,14.598220,...,17.099602,17.103828,15.443963,14.858520,16.368866,16.734613,15.965132,17.293944,15.537585,15.507849
gene3,13.436280,13.307969,14.890846,13.204174,14.829179,13.486802,15.378179,15.643631,15.559939,14.162121,...,15.645884,13.830810,15.332932,10.214664,12.339119,13.814048,13.914506,13.426047,13.486958,14.067277
gene4,18.294261,17.974726,18.863140,18.533297,19.210888,17.354699,18.566309,18.038932,18.367294,18.373866,...,19.023189,18.095513,18.502857,17.698480,17.953829,18.083508,18.267023,18.284028,17.734886,17.669313
gene5,15.475809,15.509603,15.793549,17.357979,16.173133,16.892795,14.772458,16.405471,15.218902,16.224405,...,17.780534,17.035924,16.115341,17.649292,16.731436,15.530255,17.173240,15.627681,17.421071,16.014809
gene6,10.266355,15.378359,13.174639,15.130173,13.450667,13.972229,15.534299,13.441997,14.119366,10.074658,...,15.514640,10.660885,13.496431,10.214664,15.398012,14.176618,14.566583,13.426047,15.203165,16.014809
gene7,15.310749,18.125593,16.200175,16.715136,17.151107,17.354699,16.552677,17.495108,16.066899,16.051938,...,14.292247,16.018437,16.666356,15.424117,16.535516,17.381732,16.392553,18.159402,16.701971,16.545324
gene8,17.034540,17.008409,16.475809,15.926640,17.011382,17.574265,15.740750,18.503773,16.984437,16.788903,...,16.955212,16.683252,16.180929,16.518445,16.820245,17.482426,17.516542,16.885479,15.965006,18.554543
gene9,16.895712,17.831531,17.400707,16.948335,17.106019,17.574265,16.515683,17.659227,18.026257,16.303476,...,16.076519,17.168679,16.303786,17.394573,17.830972,17.774877,16.779577,16.148513,17.992486,17.190659
gene10,13.073710,15.233969,12.689213,15.130173,14.588171,16.294157,13.949336,15.763925,14.119366,14.466975,...,15.877210,12.245847,16.666356,16.047554,13.924081,12.591655,14.277076,15.352047,15.203165,14.719354


In [235]:
import sys
 
# setting path
sys.path.append('../')
from helper_functions.PCA import PC_cluster

df_PCA = PC_cluster.PC_s(y, 100, design)

In [236]:
df_PCA

,PC1,PC2,label
sample1,0.513792,2.221827,A
sample2,-0.784411,-2.277083,A
sample3,1.902896,1.219162,A
sample4,0.401074,-1.187685,A
sample5,1.319955,-0.678955,A
...,...,...,...
sample96,-0.115477,-0.292001,B
sample97,0.107602,-0.540829,B
sample98,-0.247923,-0.731064,B
sample99,-1.413403,-0.900077,B


In [118]:
# Fit the data to the design using lmFit from limma
fit = limma.lmFit(y, r_design)
# Make a contrasts matrix with the 1st and the last unique values
contrast_matrix = limma.makeContrasts(f"{r_design.colnames[0]}-{r_design.colnames[-1]}", levels=r_design)

In [119]:
# Fit the contrasts matrix to the lmFit data & calculate the bayesian fit
fit2 = limma.contrasts_fit(fit, contrast_matrix)
fit2 = limma.eBayes(fit2)

In [120]:
# topTreat the bayesian fit using the contrasts and add the genelist
r_output = limma.topTreat(fit2, coef=1, genelist=genes, number=np.Inf)
writexl.write_xlsx(r_output, "limma_output.xlsx")

'/Users/lili-la/AC_Lab/omicsprofiler/auto_example...
